In [56]:
# imports
import pandas as pd
import numpy as np

# show multiple outputs from one cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# import os
# os.getcwd()
# os.chdir('''C:/Users/Scott/Documents/DSC672/Project''')

In [75]:
# odds_18 = pd.read_excel('nfl odds 2018-19.xlsx')
odds_19 = pd.read_excel('nfl odds 2019-20.xlsx')

In [76]:
def to_date(dt):
    m = str(dt)[:-2]
    d = str(dt)[-2:]
    y = '2019'
    if int(m) < 3:
        y = '2020'
    date = y + m + d
    return pd.to_datetime(date, format='%Y%m%d')

In [77]:
odds_19['Date'] = odds_19['Date'].apply(to_date)

In [78]:
def to_abbr(tm):
    team_dict = {
        'GreenBay' : 'GB',
        'Chicago' : 'CHI',
        'Atlanta' : 'ATL',
        'Minnesota' : 'MIN',
        'Washington' : 'WAS',
        'Philadelphia' : 'PHI',
        'Buffalo' : 'BUF',
        'NYJets' : 'NYJ',
        'Baltimore' : 'BAL',
        'Miami' : 'MIA',
        'SanFrancisco' : 'SF',
        'TampaBay' : 'TB',
        'KansasCity' : 'KC',
        'Jacksonville' : 'JAX',
        'Tennessee' : 'TEN',
        'Cleveland' : 'CLE',
        'LARams' : 'LA',
        'Carolina' : 'CAR',
        'Detroit' : 'DET',
        'Arizona' : 'ARI',
        'Cincinnati' : 'CIN',
        'Seattle' : 'SEA',
        'Indianapolis' : 'IND',
        'LAChargers' : 'LAC',
        'NYGiants' : 'NYG',
        'Dallas' : 'DAL',
        'Pittsburgh' : 'PIT',
        'NewEngland' : 'NE',
        'Houston' : 'HOU',
        'NewOrleans' : 'NO',
        'Denver' : 'DEN',
        'Oakland' : 'OAK'
    }
    return team_dict[tm]

In [79]:
odds_19['Team'] = odds_19['Team'].apply(to_abbr)

In [80]:
odds_19.loc[(odds_19['Close'] == 'pk'), 'Close'] = 0

In [81]:
odds_19[['Close', 'ML']] = odds_19[['Close', 'ML']].astype(float)

In [82]:
pr_pts = []
wps = []
for idx, row in odds_19.iterrows():
    if row['ML'] > 0:
        total = row['Close']
        if row['VH'] == 'V':
            spd = odds_19.iloc[idx+1]['Close']
        else:
            spd = odds_19.iloc[idx-1]['Close']
        proj_pts = (total - spd) / 2
        win_pb = 100 / (row['ML'] + 100)
    else:
        spd = row['Close']
        if row['VH'] == 'V':
            total = odds_19.iloc[idx+1]['Close']
        else:
            total = odds_19.iloc[idx-1]['Close']
        proj_pts = (total + spd) / 2
        win_pb = -row['ML'] / (100 - row['ML'])

    pr_pts.append(proj_pts)
    wps.append(win_pb)

odds_19['Projected Pts'] = pr_pts
odds_19['Win Prob'] = wps

In [84]:
odds_19.head()

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H,Projected Pts,Win Prob
0,2019-09-05,451,V,GB,0,7,0,3,10,46,46.5,140.0,23,21.75,0.416667
1,2019-09-05,452,H,CHI,3,0,0,0,3,3.5,3.0,-160.0,3,24.75,0.615385
2,2019-09-08,453,V,ATL,0,0,0,12,12,47.5,46.5,160.0,2.5,21.50,0.384615
3,2019-09-08,454,H,MIN,14,7,7,0,28,4.5,3.5,-190.0,22.5,25.00,0.655172
4,2019-09-08,455,V,WAS,10,10,0,7,27,46.5,44.0,425.0,23,17.00,0.190476


In [85]:
pbp = pd.read_csv('https://raw.githubusercontent.com/ryurko/nflscrapR-data/master/play_by_play_data/regular_season/reg_pbp_2019.csv', low_memory=False)

In [86]:
pbp['game_date'] = pd.to_datetime(pbp['game_date'])

In [87]:
combined_df = pd.merge(pbp, odds_19[['Date', 'Team', 'Projected Pts', 'Win Prob']], left_on=['game_date', 'posteam'], right_on=['Date', 'Team'], how='left').drop(columns=['Date', 'Team'])

In [88]:
combined_df.head()

,play_id,game_id,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,...,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv,Projected Pts,Win Prob
0,35,2019090500,CHI,GB,GB,away,CHI,CHI,35.0,2019-09-05,...,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0,21.75,0.416667
1,50,2019090500,CHI,GB,GB,away,CHI,GB,75.0,2019-09-05,...,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0,21.75,0.416667
2,71,2019090500,CHI,GB,GB,away,CHI,GB,75.0,2019-09-05,...,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0,21.75,0.416667
3,95,2019090500,CHI,GB,GB,away,CHI,GB,75.0,2019-09-05,...,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0,21.75,0.416667
4,125,2019090500,CHI,GB,GB,away,CHI,GB,85.0,2019-09-05,...,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0,21.75,0.416667


In [89]:
madden_ratings = pd.read_excel('madden_nfl_20_final_ovr_ratings_1.xlsx', header=[0,1])

In [90]:
madden_ratings.head()

2019 Arizona Cardinals                        2019 Atlanta Falcons  \
                Position         Player Overall             Position   
0                     QB   Kyler Murray    75.0                   QB   
1                     QB  Brett Hundley    64.0                   QB   
2                     HB   Kenyan Drake    81.0                   QB   
3                     HB  David Johnson    77.0                   HB   
4                     HB  Chase Edmonds    74.0                   HB   

                           2019 Baltimore Ravens                              \
            Player Overall              Position              Player Overall   
0        Matt Ryan    87.0                    QB       Lamar Jackson    92.0   
1      Matt Schaub    58.0                    QB  Robert Griffin III    66.0   
2     Kurt Benkert    48.0                    QB      Trace McSorley    55.0   
3  Devonta Freeman    85.0                    HB      Mark Ingram II    86.0   
4        Ito Smith    77.0                    HB         Gus Edwards    77.0   

  2019 Buffalo Bills  ... 2019 Seattle Seahawks 2019 Tampa Bay Buccaneers  \
            Position  ...               Overall                  Position   
0                 QB  ...                  97.0                        QB   
1                 QB  ...                  57.0                        QB   
2                 HB  ...                  86.0                        QB   
3                 HB  ...                  84.0                        HB   
4                 HB  ...                  79.0                        HB   

                           2019 Tennessee Titans                          \
            Player Overall              Position          Player Overall   
0   Jameis Winston    78.0                    QB  Ryan Tannehill    80.0   
1   Blaine Gabbert    61.0                    QB  Marcus Mariota    68.0   
2     Ryan Griffin    54.0                    HB   Derrick Henry    91.0   
3  Ronald Jones II    77.0                    HB      Dion Lewis    78.0   
4    Peyton Barber    76.0                    HB  David Fluellen    57.0   

  2019 Washington Redskins                              
                  Position              Player Overall  
0                       QB  Dwayne Haskins Jr.    70.0  
1                       QB          Alex Smith    69.0  
2                       QB         Case Keenum    68.0  
3                       QB          Colt McCoy    61.0  
4                       QB        Josh Woodrum    43.0  

[5 rows x 96 columns]

In [91]:
madden_ratings.columns = madden_ratings.columns.swaplevel(0,1)


In [92]:
madden_ratings = madden_ratings.stack(level=1)

In [93]:
madden_ratings.reset_index(inplace=True)

In [94]:
madden_ratings.drop(columns=['level_0', 'Player'], inplace=True)


In [95]:
def to_abbr_long(tm):
    team_dict = {
        'Green Bay Packers' : 'GB',
        'Chicago Bears' : 'CHI',
        'Atlanta Falcons' : 'ATL',
        'Minnesota Vikings' : 'MIN',
        'Washington Redskins' : 'WAS',
        'Philadelphia Eagles' : 'PHI',
        'Buffalo Bills' : 'BUF',
        'New York Jets' : 'NYJ',
        'Baltimore Ravens' : 'BAL',
        'Miami Dolphins' : 'MIA',
        'San Francisco 49ers' : 'SF',
        'Tampa Bay Buccaneers' : 'TB',
        'Kansas City Chiefs' : 'KC',
        'Jacksonville Jaguars' : 'JAX',
        'Tennessee Titans' : 'TEN',
        'Cleveland Browns' : 'CLE',
        'Los Angeles Rams' : 'LA',
        'Carolina Panthers' : 'CAR',
        'Detroit Lions' : 'DET',
        'Arizona Cardinals' : 'ARI',
        'Cincinnati Bengals' : 'CIN',
        'Seattle Seahawks' : 'SEA',
        'Indianapolis Colts' : 'IND',
        'Los Angeles Chargers' : 'LAC',
        'New York Giants' : 'NYG',
        'Dallas Cowboys' : 'DAL',
        'Pittsburgh Steelers' : 'PIT',
        'New England Patriots' : 'NE',
        'Houston Texans' : 'HOU',
        'New Orleans Saints' : 'NO',
        'Denver Broncos' : 'DEN',
        'Oakland Raiders' : 'OAK'
    }
    return team_dict[tm]

In [96]:
team_names = madden_ratings['level_1'].str.split(" ", n= 1, expand= True)

In [97]:
madden_ratings['team'] = team_names[1]

In [98]:
madden_ratings['team'] = madden_ratings['team'].apply(to_abbr_long)

In [99]:
madden_ratings.drop(columns='level_1', inplace=True)

In [100]:
teams_best_players = madden_ratings.groupby(['team', 'Position']).max().reset_index()

In [101]:
qb_ratings = teams_best_players[teams_best_players['Position'] == 'QB'].reset_index(drop=True).drop(columns='Position').groupby('team').mean()

In [102]:
ol_ratings = teams_best_players[teams_best_players['Position'].isin(['LT', 'LG', 'C', 'RG', 'RT'])].reset_index(drop=True).drop(columns='Position').groupby('team').mean()

In [103]:
lb_ratings = teams_best_players[teams_best_players['Position'].isin(['LOLB', 'MLB', 'ROLB'])].reset_index(drop=True).drop(columns='Position').groupby('team').mean()

In [104]:
de_ratings = teams_best_players[teams_best_players['Position'].isin(['LE', 'RE'])].reset_index(drop=True).drop(columns='Position').groupby('team').mean()


In [105]:
sfty_ratings = teams_best_players[teams_best_players['Position'].isin(['SS', 'FS'])].reset_index(drop=True).drop(columns='Position').groupby('team').mean()

In [106]:
teams_best_2_players = madden_ratings.groupby(['team', 'Position']).apply(lambda x: x.sort_values(["Overall"], ascending=False)).reset_index(drop=True).groupby(['team', 'Position']).head(2)


In [107]:
rb_ratings = teams_best_2_players[teams_best_2_players['Position'] == 'HB'].reset_index(drop=True).drop(columns='Position').groupby('team').mean()

In [108]:
wr_ratings = teams_best_2_players[teams_best_2_players['Position'] == 'WR'].reset_index(drop=True).drop(columns='Position').groupby('team').mean()

In [109]:
te_ratings = teams_best_2_players[teams_best_2_players['Position'] == 'TE'].reset_index(drop=True).drop(columns='Position').groupby('team').mean()

In [110]:
dt_ratings = teams_best_2_players[teams_best_2_players['Position'] == 'DT'].reset_index(drop=True).drop(columns='Position').groupby('team').mean()

In [111]:
cb_ratings = teams_best_2_players[teams_best_2_players['Position'] == 'CB'].reset_index(drop=True).drop(columns='Position').groupby('team').mean()

In [112]:
combined_df = combined_df.merge(qb_ratings, how='left', left_on='posteam', right_index=True).rename(columns={'Overall': 'QB Rating'})

In [113]:
combined_df = combined_df.merge(ol_ratings, how='left', left_on='posteam', right_index=True).rename(columns={'Overall': 'OL Rating'})

In [114]:
combined_df = combined_df.merge(rb_ratings, how='left', left_on='posteam', right_index=True).rename(columns={'Overall': 'RB Rating'})

In [115]:
combined_df = combined_df.merge(wr_ratings, how='left', left_on='posteam', right_index=True).rename(columns={'Overall': 'WR Rating'})

In [116]:
combined_df = combined_df.merge(te_ratings, how='left', left_on='posteam', right_index=True).rename(columns={'Overall': 'TE Rating'})

In [117]:
combined_df = combined_df.merge(lb_ratings, how='left', left_on='defteam', right_index=True).rename(columns={'Overall': 'LB Rating'})

In [118]:
combined_df = combined_df.merge(dt_ratings, how='left', left_on='defteam', right_index=True).rename(columns={'Overall': 'DT Rating'})

In [119]:
combined_df = combined_df.merge(de_ratings, how='left', left_on='defteam', right_index=True).rename(columns={'Overall': 'DE Rating'})

In [120]:
combined_df = combined_df.merge(sfty_ratings, how='left', left_on='defteam', right_index=True).rename(columns={'Overall': 'SFTY Rating'})

In [121]:
combined_df = combined_df.merge(cb_ratings, how='left', left_on='defteam', right_index=True).rename(columns={'Overall': 'CB Rating'})

In [122]:
combined_df.to_csv('nfl_scrapr_plus_plyr_odds.csv')